In [ ]:
import os
import sys
import time
import numpy as np
import pandas as pd

import anndata as sc
import snapatac2

# sys.path.insert(0, '/u/home/f/f7xiesnm/project-zipursky/code/scroutines')
# from scroutines import basicu
import atac_utils

In [ ]:
%%time
f_rna = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/superdupermegaRNA_hasraw.h5ad' 
meta = sc.read(f_rna, backed='r').obs
meta = meta[meta['Study']=='2023 Multiome']
meta['Subclass_cure'] = meta['Subclass'].apply(lambda x: x.replace('/', ''))
meta['Sample'] = meta['Sample'].astype(str) 
meta

In [ ]:
subclass_abundance = meta.groupby(['Subclass_cure', 'Sample']).size().unstack()
subclass_abundance_pass = subclass_abundance[subclass_abundance.min(axis=1) > 10]
subclass_abundance_pass

In [ ]:
meta = meta[meta['Subclass_cure'].isin(subclass_abundance_pass.index.values)]
meta

In [ ]:
organized_sample_dict = atac_utils.get_organized_sample_dict()

In [ ]:
out_dir = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/atac_fragments/frag_bed_v2/'

In [ ]:
%%time


ti = time.time()

for cond_code, samples in organized_sample_dict.items(): 
    cond_name = atac_utils.decode_cond(cond_code)
    print(cond_code, cond_name, samples)
    
    # get sub
    metasub = meta[meta['Age']==cond_name]
    cells_rna = metasub.index.values
    
    # prep ATAC
    for sample in samples:
        print(sample, time.time()-ti)
        # atac all cells
        f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/atac_fragments/frag_snap/ATAC_{sample}.h5ad'
        adata_atac = sc.read_h5ad(f)

        cell_names_raw = adata_atac.obs.index.values
        cell_names_atac = np.char.add(cell_names_raw.astype(str), f'-{sample}-2023 Multiome-{cond_code}-0')
        adata_atac.obs.index = cell_names_atac
        
        # check compatibility
        cell_names_both = np.intersect1d(cell_names_atac, cells_rna)  # test [:100]
        print(len(cell_names_both)/len(cells_rna)*100)
        
        # assign RNA type
        adata_atac = adata_atac[cell_names_both]
        adata_atac.obs = adata_atac.obs.join(metasub['Subclass_cure'])

        # export 
        snapatac2.ex.export_fragments(adata_atac, groupby='Subclass_cure',
                                     out_dir=out_dir, prefix=f'ATAC_{sample}_', suffix='.bed') 

        # break
    # break